In [ ]:
"""
CommandCenter File
read from influx and publish to mqtt broker
"""

# Importing relevant modules
import os
from dotenv import load_dotenv
from influxdb_client import InfluxDBClient, Point, Dialect
from influxdb_client.client.write_api import ASYNCHRONOUS
import paho.mqtt.client as mqtt
import time
import json
import requests
import pandas as pd

# Load environment variables from ".env"
load_dotenv()

# InfluxDB config
BUCKET = 'fullstack-influxdb' #  bucket is a named location where time series data is stored.
url = 'https://iot-group2-service1.iotcloudserve.net/'
token ='Dwj0HPIYScc1zvkB0zHpjxIVIssU_z_-unniio7sOcZl135FZ40ONj9ZX6jgiBWqkwpOQegRAL21Ix1z86SBJw=='
org = 'Chulalongkorn'
print("connecting to",url)

client = InfluxDBClient(

    url= url,
    token= token,
    org= 'Chulalongkorn'
)
write_api = client.write_api()
query_api = client.query_api()

#"temp_sht4x": 34.64,"humid_sht4x": 36.80,"temp_bmp280": 35.90,"pressure_bmp280": 1006.34}


query2 = 'from(bucket: "fullstack-influxdb")\
    |> range(start: -1h) \
    |> filter(fn: (r) => r._measurement == "sensor_data")\
    |> filter(fn: (r) => r._field == "humid_sht4x" or r._field == "pressure_bmp280" or r._field == "temp_bmp280" or r._field == "temp_sht4x")\
    |> pivot(rowKey: ["_time"], columnKey: ["_field"], valueColumn: "_value")'


csv_result = query_api.query_csv(query2,
                                 dialect=Dialect(header=False, delimiter=",", comment_prefix="#", annotations=[],
                                                 date_time_format="RFC3339"))


#csv_result =  query_api.query_csv(query2)
df1 = pd.DataFrame(csv_result)
print("Successfully convert to DataFrame")
print(df1.iloc[:-5])
#df.to_csv('RaspberryPi/CommandCenter/12_5_12_50.csv', index=False)

client.close()


In [1]:
# display all column
pd.set_option('display.max_columns', None)
df1.head()

NameError: name 'pd' is not defined

In [ ]:
# drop column
columns_to_keep = [5, 7, 8, 9, 10]
df2 = df1[df1.columns[columns_to_keep]]
df2

In [ ]:
new_column_names = ['time', 'humid_sht4x', 'pressure_bmp280', 'temp_bmp280', 'temp_sht4x']

# Rename the columns
df3 = df2.rename(columns=dict(zip(df2.columns, new_column_names)), inplace=False)
df3

In [ ]:
# Convert 'time' column to datetime format
df3['time'] = pd.to_datetime(df3['time'])

# Format 'time' column as desired
df3['time'] = df3['time'].dt.strftime('%Y/%m/%d %H:%M:%S')
df3